In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,LSTM

In [3]:
batch_size=34
epochs=100
latent_dim=256 #latent dimensionality of the encoding space
num_samples=10000 #Number of samples to train on.
data_path='fra-eng/fra.txt'

In [5]:
#Vectorize the data
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines[: min(num_samples,len(lines)-1)]:
    input_text,target_text,_=line.split('\t')
    target_text='\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
    

In [8]:
len(target_characters)

91

In [12]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters) 
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [14]:
print('Number of samples:',len(input_texts))
print('Number of unique input tokens:',num_encoder_tokens)
print('Number of unique output tokens:',num_decoder_tokens)
print('Max sequence length of inputs:',max_encoder_seq_length)
print('Max sequence length of outputs:',max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 91
Max sequence length of inputs: 14
Max sequence length of outputs: 59


In [16]:
input_token_index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index=dict([(char,i) for i,char in enumerate(target_characters)])

In [20]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [24]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 '?': 18,
 'A': 19,
 'B': 20,
 'C': 21,
 'D': 22,
 'E': 23,
 'F': 24,
 'G': 25,
 'H': 26,
 'I': 27,
 'J': 28,
 'K': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'O': 33,
 'P': 34,
 'Q': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68,
 '\xa0': 69,
 '«': 70,
 '»': 71,
 'À': 72,
 'Ç': 73,
 'É': 74,
 'Ê': 75,
 'à': 76,
 'â': 77,
 'ç': 78,
 'è': 79,
 'é': 80,
 'ê': 81,
 'î': 82,
 'ï': 83,
 'ô': 84,
 'ù': 85,
 'û': 86,
 'œ': 87,
 '\u2009': 88,
 '’': 89,
 '\u202f': 90}

In [26]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1
    encoder_input_data[i,t+1:,input_token_index[' ']]=1
    for t,char in enumerate(target_text):
        #Decoder_target_data will be ahead by one timestamp and will include the start character
        decoder_input_data[i,t-1,target_token_index[char]]=1
    decoder_input_data[i,t+1:,target_token_index[' ']]=1
    decoder_target_data[i,t:,target_token_index[' ']]=1

In [30]:
encoder_input_data[0].shape

(14, 70)

In [34]:
#Define an input sequence and process it
encoder_inputs=Input(shape=(None, num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
#We discard 'encoder_outputs' and only keep the states.
encoder_states=[state_h,state_c]

In [36]:
#Set up the decoder, using 'decoder_state' as intial state
decoder_inputs=Input(shape=(None,num_decoder_tokens))
# We set up the decoder to return full output sentences, and to return internal states as well.
# We dont use the return states in the training model, but we will use them in reference
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs, _, _=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [38]:
# Define the model that will turn 'encoder_input_data' & 'decoder_input_data' into decoder_target_data'
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

#Run training
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)f

Epoch 1/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 58s 210ms/step - accuracy: 0.6839 - loss: 0.2401 - val_accuracy: 0.6637 - val_loss: 0.0299
Epoch 2/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 49s 206ms/step - accuracy: 0.7015 - loss: 0.0662 - val_accuracy: 0.6637 - val_loss: 0.0150
Epoch 3/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 49s 209ms/step - accuracy: 0.7025 - loss: 0.0701 - val_accuracy: 0.6637 - val_loss: 0.0555
Epoch 4/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 49s 207ms/step - accuracy: 0.7018 - loss: 0.0730 - val_accuracy: 0.6637 - val_loss: 0.0542
Epoch 5/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 49s 208ms/step - accuracy: 0.7016 - loss: 0.0719 - val_accuracy: 0.6637 - val_loss: 0.0417
Epoch 6/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 48s 204ms/step - accuracy: 0.7026 - loss: 0.0705 - val_accuracy: 0.6637 - val_loss: 0.0667
Epoch 7/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 48s 203ms/step - accuracy: 0.7021 - loss: 0.0692 - val_accuracy: 0.6637 - val_loss: 0.0458
Epoch 8/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 48s 203ms/step - accuracy: 0.7019 -

In [44]:
encoder_model=Model(encoder_inputs,encoder_states)

decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]
decoder_outputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
decoder_states=[state_h,state_c]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoder_states)

# Reverse-lookup token index to decode seqeunces backto something readable
reverse_input_char_index=dict((i,char) for char,i in input_token_index.items())
reverse_target_char_index=dict((i,char) for char,i in target_token_index.items())

def decode_sequence(input_seq):
    #Encode the input as state vectors
    states_value=encoder_model.predict(input_seq)

    #Generate empty target sequence of Length 1
    target_seq=np.zeros((1,1,num_decoder_tokens))
    #Populate the first character of target sequence with the start character.
    target_seq[0,target_token_index['\t']]=1

    #Sampling loop for a batch of sequences (to simply, here we assume the batch of 1).
    stop_condition=False
    decoded_sentence=' '
    while not stop_condition:
        output_tokens,h,c=decoder_model.predict([target_seq]+states_value)

        #Sample a token
        sampled_token_index=np.argmax(output_tokens[0,-1,:])
        sampled_char=reverse_target_char_index[sampled_token_index]

        #Exit condition: Either hit max length or find stop character.
        if(sampled_char=='\n' or len(decoded_sentence)>max_decoder_seq_length):
            stop_condition=True

        #Update the target sequence (of length 1)
        target_seq=np.zeros((1,1,num_decoder_tokens))
        target_seq[0,0,sampled_token_index]=1

        #Update states
        states_value=[h,c]
    return decoded_sentence

for seq_index in range(100):
    # Take one sequence (part of the training set) for trying out decoding 
    input_seq=encoder_input_data[seq_index:seq_index+1]
    decoded_sentence=decode_sequence(input_seq)
    print('-')
    print('Input sentence:',input_texts[seq_index])
    print('Decoded sentence:',decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 552ms/step


<IPython.core.display.Javascript object>

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 